In [51]:
import numpy as np
import pandas as pd

In [52]:
data = pd.read_csv('./r101.csv')
XCOORD, YCOORD = data['XCOORD'].iloc, data['YCOORD'].iloc
READYTIME, DUEDATE = data['READYTIME'].iloc, data['DUEDATE'].iloc
SERVICETIME = data['SERVICETIME'].iloc
print(data)

     CUSTNO  XCOORD  YCOORD  DEMAND  READYTIME  DUEDATE  SERVICETIME
0         0      35      35       0          0      230            0
1         1      41      49      10        161      171           10
2         2      35      17       7         50       60           10
3         3      55      45      13        116      126           10
4         4      55      20      19        149      159           10
..      ...     ...     ...     ...        ...      ...          ...
97       97      25      21      12        133      143           10
98       98      19      21      10         58       68           10
99       99      20      26       9         83       93           10
100     100      18      18      17        185      195           10
101     101      35      35       0          0      230            0

[102 rows x 7 columns]


In [53]:
def Dist(i, j):
    dist = np.sqrt((XCOORD[i]-XCOORD[j])**2 + (YCOORD[i]-YCOORD[j])**2)
    dist = round(dist, 2)
    return dist

def TotalDist(route):
    total_dist = 0
    for point in range(len(route)-1):
        i, j = route[point], route[point+1]
        dist = Dist(i,j)
        total_dist += dist
    total_dist = round(total_dist, 2)
    return total_dist

def IsTimeReasonable(route):
    reasonable = True
    begin_j = 0
    for point in range(len(route)-1):
        begin_i = begin_j
        i, j = route[point], route[point+1]
        arrive_time = begin_i+SERVICETIME[i]+Dist(i,j)
        if (arrive_time > DUEDATE[j]):
            reasonable = False
            break
        begin_j = max(READYTIME[j], arrive_time)
    return reasonable

def LastBeginService(route):
    begin_j = 0
    for point in range(len(route)-1):
        begin_i = begin_j
        i, j = route[point], route[point+1]
        arrive_time = begin_i+SERVICETIME[i]+Dist(i,j)
        begin_j = max(READYTIME[j], arrive_time)
    return begin_j

In [54]:
mu, alpha1, alpha2, lambda1 = 1, 0.1, 0.9, 1
unselected = [i for i in range(1,101)]
current = initial = []
while (len(unselected) != 0):
    # Establish a new route
    current.append([0,101])
    while True:
        # Find the best u
        adding = current[-1]
        max_C2, best_u, best_adding = -np.inf, None, None
        for u in unselected:
            # Find the best insertion position for u
            min_C1, best_insert = np.inf, None
            for index in range(1, len(adding)):
                new_adding = adding[:]
                new_adding.insert(index, u)
                # If the time window is not reasonable
                if IsTimeReasonable(new_adding) == False:
                    continue
                i, j = index-1, index+1
                # Compute C1
                C1_1 = Dist(i,u) + Dist(u,j) - mu*Dist(i,j)
                C1_2 = LastBeginService(new_adding[:j+1]) - LastBeginService(adding[:j])
                C1 = alpha1*C1_1 + alpha2*C1_2
                # Update the best insertion position for u
                if C1 < min_C1:
                    min_C1, best_insert = C1, new_adding[:]
            # If there is no available insertion position
            if best_insert == None:
                continue
            # Compute C2
            C2 = lambda1*Dist(0,u) - min_C1
            # Update the best u
            if C2 > max_C2:
                max_C2, best_u, best_adding = C2, u, best_insert[:]
        # If there is no available u
        if best_u == None:
            break
        # Update status of route
        current[-1] = best_adding
        unselected.remove(best_u)
# Result
final_status, final_dist = current, 0
for route in final_status:
    final_dist += TotalDist(route)

In [69]:
# Initial for local search
initial = final_status
final_dist = 0
for r in initial:
    final_dist += TotalDist(r)
print("Final", round(final_dist, 2))

Final 2037.62


In [56]:
# 2-Opt* Operator
routelist = initial[:]
while True:
    replace = False
    for routeA in routelist:
        for routeB in routelist:
            if routeA != routeB:
                for i in range(2, len(routeA)):
                    a_part1, a_part2 = routeA[:i], routeA[i:]
                    for j in range(1, len(routeB)):
                        b_part1, b_part2 = routeB[:j], routeB[j:]
                        if IsTimeReasonable(a_part1 + b_part2) and IsTimeReasonable(b_part1 + a_part2):
                            new_routeA_dist = TotalDist(a_part1 + b_part2)
                            new_routeB_dist = TotalDist(b_part1 + a_part2)
                            if (new_routeA_dist + new_routeB_dist) < (TotalDist(routeA) + TotalDist(routeB)):
                                routelist[routelist.index(routeA)] = a_part1 + b_part2
                                routelist[routelist.index(routeB)] = b_part1 + a_part2
                                replace = True
                                break
                    if replace == True:
                        break
            if replace == True:
                break
        if replace == True:
            break
    if replace == False:
        break
final_dist = 0
for r in routelist:
    final_dist += TotalDist(r)
print("Final Distance:", round(final_dist, 2))

Final Distance: 1695.38


In [57]:
# The Exchange Operator
routelist = initial[:]
while True:
    replace = False
    for routeA in routelist:
        for routeB in routelist:
            if routeA != routeB:
                for pointA in routeA[1:-1]:
                    for pointB in routeB[1:-1]:
                        i, j = routeA.index(pointA), routeB.index(pointB)
                        new_routeA = routeA[:i] + [pointB] + routeA[i+1:]
                        new_routeB = routeB[:j] + [pointA] + routeB[j+1:]
                        if IsTimeReasonable(new_routeA) and IsTimeReasonable(new_routeB):
                            new_routeA_dist = TotalDist(new_routeA)
                            new_routeB_dist = TotalDist(new_routeB)
                            if (new_routeA_dist + new_routeB_dist) < (TotalDist(routeA) + TotalDist(routeB)):
                                routelist[routelist.index(routeA)] = new_routeA
                                routelist[routelist.index(routeB)] = new_routeB
                                replace = True
                                break
                    if replace == True:
                        break
            if replace == True:
                break
        if replace == True:
            break
    if replace == False:
        break

while True:
    replace = False
    for routeA in routelist:
        for routeB in routelist:
            if routeA != routeB:
                for i in range(2, len(routeA)):
                    a_part1, a_part2 = routeA[:i], routeA[i:]
                    for j in range(1, len(routeB)):
                        b_part1, b_part2 = routeB[:j], routeB[j:]
                        if IsTimeReasonable(a_part1 + b_part2) and IsTimeReasonable(b_part1 + a_part2):
                            new_routeA_dist = TotalDist(a_part1 + b_part2)
                            new_routeB_dist = TotalDist(b_part1 + a_part2)
                            if (new_routeA_dist + new_routeB_dist) < (TotalDist(routeA) + TotalDist(routeB)):
                                routelist[routelist.index(routeA)] = a_part1 + b_part2
                                routelist[routelist.index(routeB)] = b_part1 + a_part2
                                replace = True
                                break
                    if replace == True:
                        break
            if replace == True:
                break
        if replace == True:
            break
    if replace == False:
        break

final_dist = 0
for r in routelist:
    final_dist += TotalDist(r)
print("Final Distance:", round(final_dist, 2))

Final Distance: 1669.57
